# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-10-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-10-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-11 05:26:59,33.93911,67.709953,48053,1935,38099,8019.0,Afghanistan,123.439630,4.026804
1,NaN,NaN,NaN,Albania,2020-12-11 05:26:59,41.15330,20.168300,46061,965,23609,21487.0,Albania,1600.562930,2.095048
2,NaN,NaN,NaN,Algeria,2020-12-11 05:26:59,28.03390,1.659600,90579,2564,59135,28880.0,Algeria,206.560651,2.830678
3,NaN,NaN,NaN,Andorra,2020-12-11 05:26:59,42.50630,1.521800,7190,78,6505,607.0,Andorra,9305.636446,1.084840
4,NaN,NaN,NaN,Angola,2020-12-11 05:26:59,-11.20270,17.873900,15925,362,8679,6884.0,Angola,48.453935,2.273155


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,46516,46718,46837,46837,47072,47306,47516,47716,47851,48053
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,39014,39719,40501,41302,42148,42988,43683,44436,45188,46061
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,84152,85084,85927,86730,87502,88252,88825,89416,90014,90579


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1822,1841,1846,1846,1864,1874,1900,1906,1919,1935
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,822,839,852,870,889,905,922,936,951,965
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2447,2464,2480,2492,2501,2516,2527,2539,2554,2564


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36946,37218,37260,37260,37393,37685,37879,37920,38032,38099
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,19384,19912,20484,20974,21286,21617,22180,22527,23072,23609
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,54405,54990,55538,56079,56617,57146,57648,58146,58146,59135


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2818,2873,2893,2945,2979,3005,3043,3087,3117,3186
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,9051,9163,9341,9501,9626,9728,9821,9974,10087,10288
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1189,1206,1214,1217,1219,1223,1224,1240,1245,1258


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,42,42,42,42,42,42,42,42,42,42
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,137,137,137,138,138,138,138,141,141,141
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,11,29,29,29,29,29,29,30,30,30


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2020-12-11 05:26:59,32.539527,-86.644082,3186,42,0,3144.0,"Autauga, Alabama, US",5702.625785,1.318267
686,1075.0,Lamar,Alabama,US,2020-12-11 05:26:59,33.779950,-88.096680,806,8,0,798.0,"Lamar, Alabama, US",5838.464325,0.992556
687,1077.0,Lauderdale,Alabama,US,2020-12-11 05:26:59,34.901719,-87.656247,4917,60,0,4857.0,"Lauderdale, Alabama, US",5302.548286,1.220256


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,15611014,292141,5985047
India,9796744,142185,9290809
Brazil,6781799,179765,6043219


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,15611014,292141,5985047,9333826,2020-12-11 05:26:59,37.936303,-91.379001
India,9796744,142185,9290809,363750,2020-12-11 05:26:59,23.088275,81.806127
Brazil,6781799,179765,6043219,558815,2020-12-11 05:26:59,-12.669522,-48.480493
Russia,2546113,44769,2015137,486207,2020-12-11 05:26:59,54.546312,62.120860
France,2391643,57044,180348,2154251,2020-12-11 05:26:59,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,1482551,20603,0
Texas,1374256,23897,0
Florida,1094697,19591,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,1482551,20603,0,1461948,2020-12-11 05:26:59,37.843962,-120.728594
Texas,1374256,23897,0,1350359,2020-12-11 05:26:59,31.660643,-98.653069
Florida,1094697,19591,0,1075106,2020-12-11 05:26:59,28.940755,-82.700744
Illinois,823531,14844,0,808687,2020-12-11 05:26:59,39.843618,-89.178330
New York,743242,35266,0,707976,2020-12-11 05:26:59,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,487917,8149,0
Illinois,Cook,342654,7203,0
Florida,Miami-Dade,251283,3943,0
Arizona,Maricopa,241880,4244,0
Texas,Harris,202861,3110,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,487917,8149,0,479768,2020-12-11 05:26:59,34.308284,-118.228241,6037.0
Illinois,Cook,342654,7203,0,335451,2020-12-11 05:26:59,41.841448,-87.816588,17031.0
Florida,Miami-Dade,251283,3943,0,247340,2020-12-11 05:26:59,25.611236,-80.551706,12086.0
Arizona,Maricopa,241880,4244,0,237636,2020-12-11 05:26:59,33.348359,-112.491815,4013.0
Texas,Harris,202861,3110,0,199751,2020-12-11 05:26:59,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,47716,44436,89416,7127,15729,146,1469919,142928,27993,308070,...,1754911,7806,74206,1,105384,1377,101109,2078,17963,10912
2020-12-09,47851,45188,90014,7162,15804,146,1475222,144066,28000,311002,...,1771545,8104,74352,1,105852,1381,102992,2079,18062,11007
2020-12-10,48053,46061,90579,7190,15925,146,1482216,145240,28011,313688,...,1792611,8487,74498,1,106280,1385,104879,2081,18091,11081


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,1906,936,2539,78,355,4,40009,2372,908,4002,...,62130,86,611,0,928,35,867,606,364,303
2020-12-09,1919,951,2554,78,358,4,40222,2393,908,4056,...,62663,87,611,0,933,35,890,606,364,304
2020-12-10,1935,965,2564,78,362,4,40431,2416,908,4163,...,63179,90,611,0,938,35,910,606,364,305


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,37920,22527,58146,6367,8470,134,1305587,119147,25651,262314,...,3765,5415,71482,1,100476,1224,75257,1382,17268,9062
2020-12-09,38032,23072,58146,6452,8579,138,1311488,120532,25664,265815,...,3778,5564,71740,1,100952,1225,76893,1383,17295,9147
2020-12-10,38099,23609,59135,6505,8679,138,1318187,121882,25665,269665,...,3787,5747,71740,1,101384,1225,78715,1383,17307,9253


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2818,2873,2893,2945,2979,3005,3043,3087,3117,3186
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,701,713,725,734,744,755,759,769,786,806
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,4267,4360,4433,4522,4593,4626,4662,4722,4805,4917
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1334,1353,1373,1399,1421,1422,1431,1453,1480,1525
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7872,7925,8015,8071,8131,8171,8218,8298,8386,8499


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-08,276665,38116,0,378157,174325,1415396,268589,138258,41464,49,...,414749,1346643,219971,5180,1680,262730,187327,57060,448009,37475
2020-12-09,280187,38707,0,382601,176652,1448987,272346,140548,42393,49,...,422962,1359951,222545,5285,1698,267128,189863,58462,452180,37885
2020-12-10,284922,39345,0,387529,178854,1482551,276995,142979,43036,49,...,428973,1374256,225946,5413,1733,271043,192413,59695,456889,38223


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-08        3087    9974    1240  1317   3426     719   1257    7300   
2020-12-09        3117   10087    1245  1322   3496     722   1263    7392   
2020-12-10        3186   10288    1258  1359   3600     722   1287    7534   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-08         1945     1125  ...    1521    369     2213      516   
2020-12-09         1961     1134  ...    1552    375     2237      526   
2020-12-10         1977     1149  ...    1574    373     2267      534   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-08           2200  1822  1237          0      577    425  
2020-12-09           2245  1834  1250          0      586    427  
2020-12-10           2281  1852  1271          0      596    428  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-08,3942,146,0,6973,2752,20205,3372,5242,803,0,...,5109,23372,972,85,23,4260,2967,870,4054,289
2020-12-09,3985,150,0,7081,2786,20411,3639,5285,803,0,...,5171,23657,995,86,23,4281,3016,901,4142,299
2020-12-10,4034,155,0,7154,2820,20603,3759,5327,806,0,...,5240,23897,1016,89,23,4335,2850,921,4209,299


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-08          42     141      30   39     46      20     42     127   
2020-12-09          42     141      30   39     47      20     42     127   
2020-12-10          42     141      30   40     47      20     42     129   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-08           54       24  ...       7      8       12        3   
2020-12-09           55       24  ...       7      8       13        4   
2020-12-10           55       24  ...       7      8       13        4   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-08             11     2     6          9       10      2  
2020-12-09             13     2     7          0       11      2  
2020-12-10             13     2     7          0       11      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,0.004209,0.017238,0.006654,0.006070,0.005176,0.0,0.002462,0.004103,0.000214,0.007776,...,0.007108,0.040107,0.002066,0.0,0.004576,0.007315,0.013543,-0.127990,0.001785,0.006735
2020-12-09,0.002829,0.016923,0.006688,0.004911,0.004768,0.0,0.003608,0.007962,0.000250,0.009517,...,0.009479,0.038176,0.001967,0.0,0.004441,0.002905,0.018623,0.000481,0.005511,0.008706
2020-12-10,0.004221,0.019319,0.006277,0.003910,0.007656,0.0,0.004741,0.008149,0.000393,0.008637,...,0.011891,0.047261,0.001964,0.0,0.004043,0.002896,0.018322,0.000962,0.001606,0.006723


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,0.003158,0.015184,0.004749,0.0,0.002825,0.0,0.003033,0.011945,0.0,0.026944,...,0.009735,0.036145,0.0,NaN,0.004329,0.0,0.022406,-0.066256,0.0,0.030612
2020-12-09,0.006821,0.016026,0.005908,0.0,0.008451,0.0,0.005324,0.008853,0.0,0.013493,...,0.008579,0.011628,0.0,NaN,0.005388,0.0,0.026528,0.000000,0.0,0.003300
2020-12-10,0.008338,0.014721,0.003915,0.0,0.011173,0.0,0.005196,0.009611,0.0,0.026381,...,0.008235,0.034483,0.0,NaN,0.005359,0.0,0.022472,0.000000,0.0,0.003289


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,0.001082,0.015645,0.008639,0.011759,0.014007,0.000000,0.003760,0.012733,0.000195,0.021428,...,0.004804,0.033397,0.002342,0.0,0.004941,0.000000,0.016259,-0.110682,0.003312,0.010031
2020-12-09,0.002954,0.024193,0.000000,0.013350,0.012869,0.029851,0.004520,0.011624,0.000507,0.013347,...,0.003453,0.027516,0.003609,0.0,0.004737,0.000817,0.021739,0.000724,0.001564,0.009380
2020-12-10,0.001762,0.023275,0.017009,0.008215,0.011656,0.000000,0.005108,0.011200,0.000039,0.014484,...,0.002382,0.032890,0.000000,0.0,0.004279,0.000000,0.023695,0.000000,0.000694,0.011588


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-08,0.016295,0.015371,NaN,0.033659,0.013270,0.017664,0.015007,0.017770,0.018496,0.0,...,0.014726,0.011207,0.010720,0.019685,0.018799,0.014911,0.015851,0.016605,0.010420,0.013249
2020-12-09,0.012730,0.015505,NaN,0.011752,0.013349,0.023733,0.013988,0.016563,0.022405,0.0,...,0.019802,0.009882,0.011702,0.020270,0.010714,0.016740,0.013538,0.024571,0.009310,0.010941
2020-12-10,0.016899,0.016483,NaN,0.012880,0.012465,0.023164,0.017070,0.017297,0.015168,0.0,...,0.014212,0.010519,0.015282,0.024219,0.020612,0.014656,0.013431,0.021091,0.010414,0.008922


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-08      0.014459  0.015579  0.013072  0.013857  0.030686  0.007003   
2020-12-09      0.009718  0.011329  0.004032  0.003797  0.020432  0.004172   
2020-12-10      0.022137  0.019927  0.010442  0.027988  0.029748  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-08      0.010450  0.018842  0.015666  0.013514  ...  0.015354   
2020-12-09      0.004773  0.012603  0.008226  0.008000  ...  0.020381   
2020-12-10      0.019002  0.019210  0.008159  0.013228  ...  0.014175   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-08      0.019337  0.006824  0.015748   0.011029  0.006630  0.013104   
2020-12-09      0.016260  0.010845  0.019380   0.020455  0.006586  0.010509   
2020-12-10     -0.005333  0.013411  0.015209   0.016036  0.009815  0.016800   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-08            NaN  0.037770  0.007109  
2020-12-09            NaN  0.015598  0.004706  
2020-12-10            NaN  0.017065  0.002342  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-08,0.012847,0.000000,NaN,0.003309,0.014375,0.009493,0.004169,0.003446,0.012610,NaN,...,0.019964,0.008022,0.024236,0.049383,0.0,0.012357,0.008841,0.034483,0.020388,0.000000
2020-12-09,0.010908,0.027397,NaN,0.015488,0.012355,0.010195,0.079181,0.008203,0.000000,NaN,...,0.012135,0.012194,0.023663,0.011765,0.0,0.004930,0.016515,0.035632,0.021707,0.034602
2020-12-10,0.012296,0.033333,NaN,0.010309,0.012204,0.009407,0.032976,0.007947,0.003736,NaN,...,0.013344,0.010145,0.021106,0.034884,0.0,0.012614,-0.055040,0.022198,0.016176,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin   Barbour      Bibb    Blount Bullock Butler   
2020-12-08         0.0  0.021739  0.034483  0.000000  0.000000     0.0    0.0   
2020-12-09         0.0  0.000000  0.000000  0.000000  0.021739     0.0    0.0   
2020-12-10         0.0  0.000000  0.000000  0.025641  0.000000     0.0    0.0   

Province_State                               ... Wyoming                   \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte  Sheridan   
2020-12-08      0.000000  0.000000      0.0  ...     0.0    0.0  0.000000   
2020-12-09      0.000000  0.018519      0.0  ...     0.0    0.0  0.083333   
2020-12-10      0.015748  0.000000      0.0  ...     0.0    0.0  0.000000   

Province_State                                                                  
Admin2          Sublette Sweetwater Teton     Uinta Unassigned Washakie Weston  
2020-12-08      0.000000   0.000000   0.0  0.000000        0.0      0.0    0.0  
2020-12-09      0.333333   0.181818   0.0  0.166667       -1.0      0.1    0.0  
2020-12-10      0.000000   0.000000   0.0  0.000000        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,0.004259,0.017652,0.007204,0.005982,0.004505,0.003611,0.002593,0.004736,0.000313,0.008439,...,0.008123,0.037611,0.001728,0.0,0.004119,0.004213,0.016110,-0.055611,0.001647,0.008724
2020-12-09,0.003544,0.017287,0.006946,0.005446,0.004636,0.001806,0.003100,0.006349,0.000282,0.008978,...,0.008801,0.037893,0.001848,0.0,0.004280,0.003559,0.017366,-0.027565,0.003579,0.008715
2020-12-10,0.003883,0.018303,0.006611,0.004678,0.006146,0.000903,0.003921,0.007249,0.000337,0.008807,...,0.010346,0.042577,0.001906,0.0,0.004162,0.003228,0.017844,-0.013301,0.002592,0.007719


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,0.006551,0.017088,0.004808,0.001017,0.001788,9.934107e-09,0.003195,0.010427,1.076695e-06,0.023751,...,0.007034,0.023745,0.000014,NaN,0.004411,2.509858e-31,0.022235,-0.027170,0.000613,0.020411
2020-12-09,0.006686,0.016557,0.005358,0.000509,0.005119,4.967054e-09,0.004259,0.009640,5.383476e-07,0.018622,...,0.007806,0.017686,0.000007,NaN,0.004900,1.254929e-31,0.024382,-0.013585,0.000306,0.011856
2020-12-10,0.007512,0.015639,0.004637,0.000254,0.008146,2.483527e-09,0.004728,0.009626,2.691738e-07,0.022501,...,0.008020,0.026085,0.000004,NaN,0.005129,6.274644e-32,0.023427,-0.006793,0.000153,0.007573


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-08,0.003128,0.018805,0.008916,0.010547,0.008455,0.002061,0.004275,0.010465,0.000183,0.019295,...,0.006610,0.031554,0.001911,0.0,0.004730,0.001306,0.017574,-0.053539,0.002458,0.008865
2020-12-09,0.003041,0.021499,0.004458,0.011949,0.010662,0.015956,0.004398,0.011045,0.000345,0.016321,...,0.005032,0.029535,0.002760,0.0,0.004734,0.001062,0.019657,-0.026408,0.002011,0.009122
2020-12-10,0.002401,0.022387,0.010734,0.010082,0.011159,0.007978,0.004753,0.011122,0.000192,0.015402,...,0.003707,0.031213,0.001380,0.0,0.004506,0.000531,0.021676,-0.013204,0.001352,0.010355


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-08,0.013082,0.016730,NaN,0.021986,0.011243,0.018312,0.015514,0.026106,0.019312,1.435533e-81,...,0.014910,0.009664,0.011563,0.019880,0.013318,0.014736,0.020226,0.019832,0.009038,0.014034
2020-12-09,0.012906,0.016118,NaN,0.016869,0.012296,0.021022,0.014751,0.021335,0.020859,7.177667e-82,...,0.017356,0.009773,0.011632,0.020075,0.012016,0.015738,0.016882,0.022201,0.009174,0.012487
2020-12-10,0.014903,0.016300,NaN,0.014874,0.012381,0.022093,0.015911,0.019316,0.018013,3.588833e-82,...,0.015784,0.010146,0.013457,0.022147,0.016314,0.015197,0.015156,0.021646,0.009794,0.010704


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-08      0.013113  0.013369  0.007590  0.010595  0.020139  0.004213   
2020-12-09      0.011415  0.012349  0.005811  0.007196  0.020286  0.004193   
2020-12-10      0.016776  0.016138  0.008127  0.017592  0.025017  0.002096   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-08      0.009194  0.014990  0.010330  0.012184  ...  0.016325   
2020-12-09      0.006983  0.013796  0.009278  0.010092  ...  0.018353   
2020-12-10      0.012993  0.016503  0.008719  0.011660  ...  0.016264   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-08      0.015849  0.009795  0.016910   0.016392  0.011603  0.014607   
2020-12-09      0.016054  0.010320  0.018145   0.018423  0.009095  0.012558   
2020-12-10      0.005361  0.011865  0.016677   0.017229  0.009455  0.014679   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-08           -1.0  0.032265  0.005439  
2020-12-09           -1.0  0.023931  0.005073  
2020-12-10           -1.0  0.020498  0.003707  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-08,0.008709,0.009642,NaN,0.003049,0.015567,0.007114,0.003682,0.005908,0.008245,NaN,...,0.015735,0.006259,0.016442,0.034146,2.015851e-12,0.007602,0.006552,0.022826,0.014809,0.023824
2020-12-09,0.009809,0.018519,NaN,0.009269,0.013961,0.008655,0.041432,0.007055,0.004122,NaN,...,0.013935,0.009227,0.020052,0.022955,1.007926e-12,0.006266,0.011534,0.029229,0.018258,0.029213
2020-12-10,0.011052,0.025926,NaN,0.009789,0.013082,0.009031,0.037204,0.007501,0.003929,NaN,...,0.013639,0.009686,0.020579,0.028919,5.039628e-13,0.009440,-0.021753,0.025713,0.017217,0.014607


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-08      0.000006  0.012662  0.030123  0.015489  0.002015  0.001645   
2020-12-09      0.000003  0.006331  0.015062  0.007745  0.011877  0.000822   
2020-12-10      0.000002  0.003166  0.007531  0.016693  0.005939  0.000411   

Province_State                                              ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...      Park   
2020-12-08      0.000024  0.001809  0.002154  5.086264e-07  ...  0.041674   
2020-12-09      0.000012  0.000905  0.010337  2.543132e-07  ...  0.020837   
2020-12-10      0.000006  0.008326  0.005168  1.271566e-07  ...  0.010419   

Province_State                                                         \
Admin2            Platte  Sheridan  Sublette Sweetwater         Teton   
2020-12-08      0.000571  0.024290  0.140625   0.035417  1.455192e-11   
2020-12-09      0.000286  0.053812  0.236979   0.108618  7.275958e-12   
2020-12-10      0.000143  0.026906  0.118490   0.054309  3.637979e-12   

Province_State                                              
Admin2             Uinta    Unassigned  Washakie    Weston  
2020-12-08      0.053906 -4.840016e-10  0.062509  0.003906  
2020-12-09      0.110286 -5.019608e-01  0.081254  0.001953  
2020-12-10      0.055143 -5.019608e-01  0.040627  0.000977  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201210,AK,37962,NaN,1075398,NaN,totalTestsViral,1113360,164.0,860.0,...,1113360,5,13,68dbda152af1d2b3fbf2d86cea850202c1cb9c81,0,0,0,0,0,NaN
1,20201210,AL,284922,49930.0,1449370,NaN,totalTestsPeopleViral,1684362,2170.0,27618.0,...,1734292,49,0,379514f61ccd2e045bafed7ea1f4d4b0d0554916,0,0,0,0,0,NaN
2,20201210,AR,178854,24781.0,1658925,NaN,totalTestsViral,1812998,1005.0,9709.0,...,1837779,34,34,e78028d8355d6ad02f39330283eafce982d25519,0,0,0,0,0,NaN
3,20201210,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,b8abaeca231968c80a76ac13bc1bb8089eb7ffb5,0,0,0,0,0,NaN
4,20201210,AZ,387529,14102.0,2064282,NaN,totalTestsViral,4136075,3408.0,29335.0,...,2451811,73,455,0cc2594126beaf8c06839513b28304df417e6af4,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201210,AK,37962.0,NaN,1075398.0,NaN,totalTestsViral,1113360.0,164.0,860.0,...,1113360,5,13,68dbda152af1d2b3fbf2d86cea850202c1cb9c81,0,0,0,0,0,NaN
1,20201210,AL,284922.0,49930.0,1449370.0,NaN,totalTestsPeopleViral,1684362.0,2170.0,27618.0,...,1734292,49,0,379514f61ccd2e045bafed7ea1f4d4b0d0554916,0,0,0,0,0,NaN
2,20201210,AR,178854.0,24781.0,1658925.0,NaN,totalTestsViral,1812998.0,1005.0,9709.0,...,1837779,34,34,e78028d8355d6ad02f39330283eafce982d25519,0,0,0,0,0,NaN
3,20201210,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,b8abaeca231968c80a76ac13bc1bb8089eb7ffb5,0,0,0,0,0,NaN
4,20201210,AZ,387529.0,14102.0,2064282.0,NaN,totalTestsViral,4136075.0,3408.0,29335.0,...,2451811,73,455,0cc2594126beaf8c06839513b28304df417e6af4,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-10,AK,37962,NaN,1075398,NaN,totalTestsViral,1113360,164.0,860.0,NaN,...,1113360,5,13,68dbda152af1d2b3fbf2d86cea850202c1cb9c81,0,0,0,0,0,NaN
2020-12-10,AL,284922,49930.0,1449370,NaN,totalTestsPeopleViral,1684362,2170.0,27618.0,NaN,...,1734292,49,0,379514f61ccd2e045bafed7ea1f4d4b0d0554916,0,0,0,0,0,NaN
2020-12-10,AR,178854,24781.0,1658925,NaN,totalTestsViral,1812998,1005.0,9709.0,372.0,...,1837779,34,34,e78028d8355d6ad02f39330283eafce982d25519,0,0,0,0,0,NaN
2020-12-10,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,b8abaeca231968c80a76ac13bc1bb8089eb7ffb5,0,0,0,0,0,NaN
2020-12-10,AZ,387529,14102.0,2064282,NaN,totalTestsViral,4136075,3408.0,29335.0,799.0,...,2451811,73,455,0cc2594126beaf8c06839513b28304df417e6af4,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-10,AK,37962.0,NaN,1075398.0,NaN,totalTestsViral,1113360.0,164.0,860.0,NaN,...,1113360,5,13,68dbda152af1d2b3fbf2d86cea850202c1cb9c81,0,0,0,0,0,NaN
2020-12-10,AL,284922.0,49930.0,1449370.0,NaN,totalTestsPeopleViral,1684362.0,2170.0,27618.0,NaN,...,1734292,49,0,379514f61ccd2e045bafed7ea1f4d4b0d0554916,0,0,0,0,0,NaN
2020-12-10,AR,178854.0,24781.0,1658925.0,NaN,totalTestsViral,1812998.0,1005.0,9709.0,372.0,...,1837779,34,34,e78028d8355d6ad02f39330283eafce982d25519,0,0,0,0,0,NaN
2020-12-10,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,b8abaeca231968c80a76ac13bc1bb8089eb7ffb5,0,0,0,0,0,NaN
2020-12-10,AZ,387529.0,14102.0,2064282.0,NaN,totalTestsViral,4136075.0,3408.0,29335.0,799.0,...,2451811,73,455,0cc2594126beaf8c06839513b28304df417e6af4,0,0,0,0,0,NaN
